In [3]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

# Read dataset

In [4]:
sales_receipts = pd.read_csv("dataset/201904 sales reciepts.csv")

In [5]:
sales_receipts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [6]:
products = pd.read_csv("dataset/product.csv")
products.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


In [41]:
products.loc[52,:]

product_id                                                          53
product_group                                                Beverages
product_category                                                   Tea
product_type                                           Brewed Chai tea
product                                      Traditional Blend Chai Lg
product_description        Sit back and think of the tropical breezes.
unit_of_measure                                                  24 oz
current_wholesale_price                                           0.75
current_retail_price                                            $3.00 
tax_exempt_yn                                                        Y
promo_yn                                                             N
new_product_yn                                                       N
Name: 52, dtype: object

# Data Wrangling

## Merge Data

In [10]:
sales_receipts = sales_receipts[["transaction_id", "transaction_date", "sales_outlet_id", "customer_id", "product_id", "quantity"]]
products = products[["product_id", "product_category", "product"]]

dataset = pd.merge(sales_receipts, products, on="product_id", how="left")
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


## Remove sizes

In [13]:
dataset[dataset["product"].str.contains("Dark chocolate")]["product"].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [14]:
dataset["product"].nunique()

80

In [17]:
dataset["product"] = dataset["product"].str.replace(" Rg", "")
dataset["product"] = dataset["product"].str.replace(" Lg", "")
dataset["product"] = dataset["product"].str.replace(" Sm", "")

In [18]:
dataset["product"].nunique()

45

In [19]:
dataset["product"].unique()

array(['Traditional Blend Chai', 'Brazilian', 'Serenity Green Tea',
       'Our Old Time Diner Blend', 'Jamaican Coffee River', 'Ethiopia',
       'English Breakfast', 'Sustainably Grown Organic', 'Earl Grey',
       'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate',
       'Columbian Medium Roast', 'Oatmeal Scone', 'Morning Sunrise Chai',
       'Peppermint', 'Jumbo Savory Scone', 'Lemon Grass',
       'Chocolate Chip Biscotti', 'Spicy Eye Opener Chai',
       'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
       'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant',
       'Almond Croissant', 'Ginger Scone', 'Ouro Brasileiro shot',
       'Organic Decaf Blend', 'Chocolate syrup', 'Hazelnut syrup',
       'Carmel syrup', 'Sugar Free Vanilla syrup',
       'Jamacian Coffee River', 'Guatemalan Sustainably Grown',
       'Civet Cat', 'Chili Mayan', 'Primo Espresso Roast',
       'Brazilian - Organic', 'I Need My Bean! Diner mug',
       'Espresso Roast', 'I Need 

## Choose product subset

In [20]:
products_to_take = [
    'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate', 'Chocolate Chip Biscotti', 'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
    'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant', 'Almond Croissant', 'Chocolate syrup', 'Hazelnut syrup', 'Carmel syrup', 'Sugar Free Vanilla syrup',
    
]

In [21]:
dataset = dataset[dataset["product"].isin(products_to_take)]
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


In [23]:
dataset[["product", "product_category"]].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Latte,Coffee
3,Dark chocolate,Drinking Chocolate
4,Chocolate Chip Biscotti,Bakery
5,Ginger Biscotti,Bakery
6,Chocolate Croissant,Bakery
7,Hazelnut Biscotti,Bakery
8,Cranberry Scone,Bakery
9,Scottish Cream Scone,Bakery


## Clean transactions

In [24]:
dataset["transaction"] = dataset["transaction_id"].astype(str) + '_' + dataset["customer_id"].astype(str)
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [27]:
num_of_items_per_transaction = dataset["transaction"].value_counts().reset_index()
num_of_items_per_transaction.head()

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24


In [29]:
num_of_items_per_transaction[num_of_items_per_transaction["count"] == 1]

,transaction,count
2448,2553_8321,1
2449,347_5096,1
2450,404_5003,1
2451,640_5722,1
2452,341_5667,1
...,...,...
7674,382_8184,1
7675,381_8110,1
7676,378_8450,1
7677,2142_5458,1


In [31]:
valid_transactions = num_of_items_per_transaction[(num_of_items_per_transaction["count"] > 1)]["transaction"].to_list()
valid_transactions[:10]

['209_0',
 '206_0',
 '204_0',
 '208_0',
 '203_0',
 '205_0',
 '207_0',
 '301_0',
 '202_0',
 '151_0']

In [32]:
dataset = dataset[dataset["transaction"].isin(valid_transactions)]
dataset.shape

(9257, 9)

# Product Trends

In [33]:
dataset["product_category"].value_counts()

product_category
Coffee                3101
Bakery                2995
Flavours              2246
Drinking Chocolate     893
Packaged Chocolate      22
Name: count, dtype: int64

In [34]:
dataset["product"].value_counts()

product
Cappuccino                  1252
Latte                       1232
Dark chocolate               915
Chocolate Croissant          629
Espresso shot                617
Sugar Free Vanilla syrup     605
Chocolate syrup              568
Carmel syrup                 561
Hazelnut syrup               512
Chocolate Chip Biscotti      353
Croissant                    351
Scottish Cream Scone         343
Cranberry Scone              342
Almond Croissant             336
Hazelnut Biscotti            331
Ginger Biscotti              310
Name: count, dtype: int64

# Popularity Recommendation Engine

In [36]:
product_recommendation = dataset.groupby(["product", "product_category"]).count().reset_index()

In [38]:
product_recommendation = product_recommendation[["product", "product_category", "transaction_id"]]
product_recommendation = product_recommendation.rename(columns={"transaction_id":"number_of_transactions"})

In [39]:
product_recommendation.head()

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,336
1,Cappuccino,Coffee,1252
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,353
4,Chocolate Croissant,Bakery,629


In [40]:
product_recommendation.to_csv('api/recommendation_objects/popularity_recommendations.csv', index=False)

# Apriori Recommendation Engine

In [42]:
train_basket = (dataset.groupby(["transaction", "product"])["product"].count().reset_index(name="count"))
train_basket.head()

,transaction,product,count
0,1001_8306,Cappuccino,1
1,1001_8306,Carmel syrup,1
2,1002_0,Carmel syrup,1
3,1002_0,Dark chocolate,2
4,1002_0,Espresso shot,1


In [44]:
my_basket = train_basket.pivot_table(index="transaction", columns="product", values="count").fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Hazelnut Biscotti,Hazelnut syrup,Latte,Scottish Cream Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_5559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1006_0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
def encode_units(x):
    if x<=0:
        return 0
    if x>0:
        return 1

my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head(10)

/tmp/ipykernel_342759/603530051.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  my_basket_sets = my_basket.applymap(encode_units)


product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Hazelnut Biscotti,Hazelnut syrup,Latte,Scottish Cream Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1009_5183,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
100_0,0,1,1,0,0,1,0,1,1,0,0,1,0,0,1,0


In [46]:
frequent_items = apriori(my_basket_sets, min_support=0.05, use_colnames=True)
frequent_items.head()

/home/gabryel/development/ai/coffee-shop-chatbot/venv/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.120507,(Almond Croissant)
1,0.404820,(Cappuccino)
2,0.206699,(Carmel syrup)
3,0.122141,(Chocolate Chip Biscotti)
4,0.143791,(Chocolate Croissant)


In [54]:
rules_basket = association_rules(frequent_items, 1, metric="lift", min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Almond Croissant),(Cappuccino),0.120507,0.404820,0.057598,0.477966,1.180687,1.0,0.008815,1.140117,0.174004,0.123144,0.122897,0.310123
1,(Cappuccino),(Almond Croissant),0.404820,0.120507,0.057598,0.142281,1.180687,1.0,0.008815,1.025386,0.257125,0.123144,0.024757,0.310123
2,(Dark chocolate),(Almond Croissant),0.277369,0.120507,0.062092,0.223859,1.857647,1.0,0.028667,1.133161,0.638894,0.184915,0.117513,0.369556
3,(Almond Croissant),(Dark chocolate),0.120507,0.277369,0.062092,0.515254,1.857647,1.0,0.028667,1.490742,0.524944,0.184915,0.329193,0.369556
4,(Latte),(Almond Croissant),0.403186,0.120507,0.058824,0.145897,1.210695,1.0,0.010237,1.029727,0.291595,0.126538,0.028869,0.317016


In [57]:
rules_basket[rules_basket["antecedents"] == {"Latte"}].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
76,(Latte),(Sugar Free Vanilla syrup),0.403186,0.216503,0.117647,0.291793,1.347755,1.0,0.030356,1.106311,0.432338,0.234337,0.096095,0.417595
30,(Latte),(Carmel syrup),0.403186,0.206699,0.116422,0.288754,1.396975,1.0,0.033083,1.115367,0.476141,0.235927,0.103434,0.425997
44,(Latte),(Chocolate syrup),0.403186,0.204248,0.111520,0.276596,1.354213,1.0,0.029169,1.100010,0.438267,0.224876,0.090917,0.411298
72,(Latte),(Hazelnut syrup),0.403186,0.185458,0.109477,0.271530,1.464108,1.0,0.034703,1.118155,0.531138,0.228474,0.105670,0.430919
74,(Latte),(Scottish Cream Scone ),0.403186,0.123366,0.062500,0.155015,1.256547,1.0,0.012761,1.037455,0.342097,0.134683,0.036103,0.330819
52,(Latte),(Croissant),0.403186,0.121732,0.061683,0.152989,1.256768,1.0,0.012602,1.036903,0.342331,0.133157,0.035589,0.329850
38,(Latte),(Chocolate Croissant),0.403186,0.143791,0.059641,0.147923,1.028737,1.0,0.001666,1.004850,0.046806,0.122381,0.004826,0.281348
4,(Latte),(Almond Croissant),0.403186,0.120507,0.058824,0.145897,1.210695,1.0,0.010237,1.029727,0.291595,0.126538,0.028869,0.317016
68,(Latte),(Ginger Biscotti),0.403186,0.113154,0.058415,0.144883,1.280414,1.0,0.012793,1.037106,0.366953,0.127565,0.035778,0.330564
48,(Latte),(Cranberry Scone),0.403186,0.122141,0.055556,0.137791,1.128137,1.0,0.006310,1.018152,0.190316,0.118261,0.017828,0.296320


## Save to JSON format

In [62]:
product_categories = dataset[["product", "product_category"]].drop_duplicates().set_index("product").to_dict()["product_category"]
product_categories

{'Cappuccino': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Latte': 'Coffee',
 'Croissant': 'Bakery',
 'Scottish Cream Scone ': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ginger Biscotti': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours',
 'Sugar Free Vanilla syrup': 'Flavours'}

In [63]:
recommendations_json = {}

antecedents = rules_basket["antecedents"].unique()

for antecedent in antecedents:
    df_rec = rules_basket[rules_basket["antecedents"] == antecedent]
    df_rec = df_rec.sort_values("confidence", ascending=False)

    key = "_".join(antecedent)
    recommendations_json[key] = []

    for _, row in df_rec.iterrows():
        rec_objects = row["consequents"]
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object["product"]:
                    already_exists = True
            if already_exists:
                continue
            rec = {
                "product": rec_object,
                "product_category": product_categories[rec_object],
                "confidence": row["confidence"]
            }

            recommendations_json[key].append(rec)

In [64]:
import pprint
pprint.pp(recommendations_json)

{'Almond Croissant': [{'product': 'Dark chocolate',
                       'product_category': 'Packaged Chocolate',
                       'confidence': 0.5152542372881356},
                      {'product': 'Latte',
                       'product_category': 'Coffee',
                       'confidence': 0.48813559322033895},
                      {'product': 'Cappuccino',
                       'product_category': 'Coffee',
                       'confidence': 0.4779661016949152}],
 'Cappuccino': [{'product': 'Sugar Free Vanilla syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.30171543895055497},
                {'product': 'Chocolate syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.2926337033299697},
                {'product': 'Carmel syrup',
                 'product_category': 'Flavours',
                 'confidence': 0.27447023208879917},
                {'product': 'Hazelnut syrup',
            

In [65]:
import json

with open("api/recommendation_objects/apriori_recommendations.json", 'w') as json_file:
    json.dump(recommendations_json, json_file)